### Security & Role-Based Access Control (RBAC) using:

* Ollama (for running a local LLM like llama3)

* FastAPI (for building a lightweight API backend)

* Basic RBAC logic (custom roles + access control)



Simulate Role-Based Access Control where the user's role (e.g., admin, analyst, or viewer) determines what information or instructions the LLM can access or respond with. This is useful in:

* Enterprise security assistants

* Internal chatbots with access restrictions

* Data governance over LLM usage

**Model Context Protocol (MCP)** is a concept to structure and control what context is injected into prompts — especially critical for access-controlled environments.

### Overview
A FastAPI server with `/chat` endpoint

Each request includes: `user_input` + `user_role`

The backend constructs a role-specific prompt using MCP-style context injection

The Ollama model (llama3) runs locally and responds with scoped knowledge

#### Requirements
```bash
pip install fastapi uvicorn ollama
ollama pull llama3
```

#### main.py

In [ ]:
from fastapi import FastAPI, HTTPException
from pydantic import BaseModel
import ollama

app = FastAPI()

MODEL_NAME = "llama3"

# Define allowed roles
ROLE_CONTEXTS = {
    "admin": "You are speaking to an admin. You can provide system-level insights and internal policies.",
    "analyst": "You are speaking to a security analyst. Answer only with threat detection and incident insights.",
    "viewer": "You are speaking to a general viewer. Avoid exposing sensitive or internal data."
}

# Pydantic model for request
class ChatRequest(BaseModel):
    user_input: str
    user_role: str

@app.post("/chat")
def chat_endpoint(payload: ChatRequest):
    role = payload.user_role.lower()
    if role not in ROLE_CONTEXTS:
        raise HTTPException(status_code=403, detail="Unauthorized role.")

    # Inject context (MCP-style)
    system_prompt = ROLE_CONTEXTS[role]
    messages = [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": payload.user_input}
    ]

    try:
        response = ollama.chat(model=MODEL_NAME, messages=messages)
        return {"response": response['message']['content']}
    except Exception as e:
        raise HTTPException(status_code=500, detail=str(e))


### Run the server
```bash
uvicorn main:app --reload
```

#### Example call with `curl`

```bash
curl -X POST "http://127.0.0.1:8000/chat" \
  -H "Content-Type: application/json" \
  -d '{"user_input": "What are the open incidents?", "user_role": "analyst"}'

```

Why this is MVP-ready

✅ Secure Role Check (viewer vs admin)

✅ MCP-style prompt context (per role)

✅ Modular & extensible (easy to plug into frontend/UI)

✅ Works with Ollama locally (offline-friendly)

